![Hand with calculator](calculator.jpg)

Did you know that the average return from investing in stocks is [10% per year](https://www.nerdwallet.com/article/investing/average-stock-market-return) (not accounting for inflation)? But who wants to be average?! 

You have been asked to support an investment firm by analyzing trends in high-growth companies. They are interested in understanding which industries are producing the highest valuations and the rate at which new high-value companies are emerging. Providing them with this information gives them a competitive insight as to industry trends and how they should structure their portfolio looking forward.

You have been given access to their `unicorns` database, which contains the following tables:

## dates
| Column       | Description                                  |
|------------- |--------------------------------------------- |
| `company_id`   | A unique ID for the company.                 |
| `date_joined` | The date that the company became a unicorn.  |
| `year_founded` | The year that the company was founded.       |

## funding
| Column           | Description                                  |
|----------------- |--------------------------------------------- |
| `company_id`       | A unique ID for the company.                 |
| `valuation`        | Company value in US dollars.                 |
| `funding`          | The amount of funding raised in US dollars.  |
| `select_investors` | A list of key investors in the company.      |

## industries
| Column       | Description                                  |
|------------- |--------------------------------------------- |
| `company_id`   | A unique ID for the company.                 |
| `industry`     | The industry that the company operates in.   |

## companies
| Column       | Description                                       |
|------------- |-------------------------------------------------- |
| `company_id`   | A unique ID for the company.                      |
| `company`      | The name of the company.                          |
| `city`         | The city where the company is headquartered.      |
| `country`      | The country where the company is headquartered.   |
| `continent`    | The continent where the company is headquartered. |


# The output

Your query should return a table in the following format:
| industry  | year | num\_unicorns       | average\_valuation\_billions |
| --------- | ---- | ------------------- | ---------------------------- |
| industry1 | 2021 |        ---          |             ---              |
| industry2 | 2020 |        ---          |             ---              |
| industry3 | 2019 |        ---          |             ---              |
| industry1 | 2021 |        ---          |             ---              |
| industry2 | 2020 |        ---          |             ---              |
| industry3 | 2019 |        ---          |             ---              |
| industry1 | 2021 |        ---          |             ---              |
| industry2 | 2020 |        ---          |             ---              |
| industry3 | 2019 |        ---          |             ---              |

Where `industry1`, `industry2`, and `industry3` are the three top-performing industries.

# The Work
## Step 1 - Looking at each column from the tables
Despite the table schema already provided above, it's also good practice to look at how the actual data on the table look. To do that, we showed the first 5 rows of each tables by selecting all columns and limiting the output to 5.

In [6]:
-- Showing the dates table data
SELECT * FROM dates LIMIT 5;

,company_id,date_joined,year_founded
0,189,2017-06-24 00:00:00+00:00,1919
1,848,2021-06-01 00:00:00+00:00,2019
2,556,2022-02-15 00:00:00+00:00,2011
3,999,2021-11-17 00:00:00+00:00,2020
4,396,2021-10-21 00:00:00+00:00,2021


In [7]:
-- Showing the funding table data
SELECT * FROM funding LIMIT 5;

,company_id,valuation,funding,select_investors
0,189,4000000000,0,EQT Partners
1,848,1000000000,100000000,"Dragonfly Captial, Qiming Venture Partners, DS..."
2,556,2000000000,100000000,"Blackstone, Bessemer Venture Partners"
3,999,1000000000,100000000,"Goldman Sachs Asset Management, 3L"
4,396,2000000000,100000000,"Insight Partners, Softbank Group, Connect Vent..."


In [8]:
-- Showing the industries table data
SELECT * FROM industries LIMIT 5;

,company_id,industry
0,189,Health
1,848,Fintech
2,556,Internet software & services
3,999,Internet software & services
4,396,Fintech


In [1]:
-- Showing the companies table data
SELECT * FROM companies LIMIT 5;

-- note that there is a null data in city column. Since we did'nt use the city column, it was left as is

,company_id,company,city,country,continent
0,189,Otto Bock HealthCare,Duderstadt,Germany,Europe
1,848,Matrixport,,Singapore,Asia
2,556,Cloudinary,Santa Clara,United States,North America
3,999,PLACE,Bellingham,United States,North America
4,396,candy.com,New York,United States,North America


## Step 2 - Get the industry rank based on the total number of new unicorns from 2019 - 2021
To do that, two table were merged, the industries table to get the name of the industries and the dates table to filter the analysis just for the newcomers in 2019, 2020, and 2021. To rank the Industry, rank() window function is used over the number of unique id in each industries.
 
note : this query will act as a CTE in the full query that will be done in the later step. For this step, we do this query independently to show the result of this query.  

In [2]:
SELECT industry, 
	COUNT(DISTINCT company_id) AS newcomer_number,
	RANK() OVER(ORDER BY COUNT(DISTINCT company_id) DESC)
FROM dates
JOIN industries
	USING(company_id)
WHERE EXTRACT(year FROM date_joined) IN (2019,2020,2021)
GROUP BY industry

,industry,newcomer_number,rank
0,Fintech,173,1
1,Internet software & services,152,2
2,E-commerce & direct-to-consumer,75,3
3,Artificial intelligence,53,4
4,Health,52,5
5,Other,41,6
6,Cybersecurity,38,7
7,"Supply chain, logistics, & delivery",35,8
8,Data management & analytics,31,9
9,Mobile & telecommunications,18,10


## Step 3 - Get the average valuations and newcomers number of each industry in each year from 2019 - 2021.
In order to do that, three table were merged using company_id column. The industries table is used to get the industry names, the funding table is used to get the average valuations, and the dates table is used to filter the years used for analysis to only in 2019, 2020, 2021

note : this query will act as a CTE in the full query that will be done in the later step. For this step, we do this query independently to show the result of this query.  

In [6]:
SELECT  industry,
		d.year,
		ROUND(AVG(f.valuation),2) AS avg_valuation_billions,
		COUNT(i.company_id) as newcomers
FROM industries AS i
JOIN (
	SELECT company_id, valuation/1000000000 AS valuation
	FROM funding
) AS f
USING(company_id)
JOIN (
	SELECT company_id, EXTRACT(year FROM date_joined) AS year
	FROM dates
) AS d
USING(company_id)
WHERE d.year IN (2019,2020,2021)
GROUP BY industry, d.year

,industry,year,avg_valuation_billions,newcomers
0,Artificial intelligence,2019,4.50,14
1,Artificial intelligence,2020,4.00,3
2,Artificial intelligence,2021,1.42,36
3,Auto & transportation,2019,4.17,6
4,Auto & transportation,2020,3.00,5
5,Auto & transportation,2021,3.75,4
6,Consumer & retail,2019,3.67,3
7,Consumer & retail,2020,15.00,1
8,Consumer & retail,2021,2.57,7
9,Cybersecurity,2019,2.25,4


## Step 4 - recreate the previous two queries as CTE and join them to match the desired output
Using WITH statement, the previous queries are converted to CTE. The query on step on is named best_industry, while the query on the step 2 is named industry_data. 

Then, the two CTE's are joined, with the best_industry CTE for filtering the best three industries and the industry_data providing the data of each best industries from 2019 - 2021. 

In [7]:
WITH best_industry AS(
	SELECT industry, 
	COUNT(DISTINCT company_id) AS newcomer_number,
	RANK() OVER(ORDER BY COUNT(DISTINCT company_id) DESC)
	FROM dates
	JOIN industries
	USING(company_id)
	WHERE EXTRACT(year FROM date_joined) IN (2019,2020,2021)
	GROUP BY industry), 
industry_data AS(
	SELECT  industry,
		d.year,
		ROUND(AVG(f.valuation),2) AS avg_valuation_billions,
		COUNT(i.company_id) as newcomers
	FROM industries AS i
	JOIN (
		SELECT company_id, valuation/1000000000 AS valuation
		FROM funding
	) AS f
	USING(company_id)
	JOIN (
		SELECT company_id, EXTRACT(year FROM date_joined) AS year
		FROM dates
	) AS d
	USING(company_id)
	WHERE d.year IN (2019,2020,2021)
	GROUP BY industry, d.year
)

SELECT industry,
		year,
		newcomers AS num_unicorns,
		avg_valuation_billions
FROM industry_data
JOIN best_industry
USING(industry)
WHERE rank IN(1,2,3)
ORDER BY rank, year DESC, num_unicorns DESC;

,industry,year,num_unicorns,avg_valuation_billions
0,Fintech,2021,138,2.75
1,Fintech,2020,15,4.33
2,Fintech,2019,20,6.80
3,Internet software & services,2021,119,2.15
4,Internet software & services,2020,20,4.35
5,Internet software & services,2019,13,4.23
6,E-commerce & direct-to-consumer,2021,47,2.47
7,E-commerce & direct-to-consumer,2020,16,4.00
8,E-commerce & direct-to-consumer,2019,12,2.58
